In [42]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

In [32]:
salary_train = pd.read_csv("salary_train.csv")
salary_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [34]:
for i in range(len(salary_train.columns)):
    salary_train.iloc[:,i] = salary_train.iloc[:,i].map(lambda x: x if type(x)!=str else x.lower())
    salary_train.iloc[:,i] = salary_train.iloc[:,i].replace('[^a-zA-Z0-9]', ' ', regex = True)
salary_train

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,NaN,20355
...,...,...,...,...
59995,as a result of continued growth first class s...,whitley bay,contract,26400
59996,php mvc web developer macclesfieldcirca ...,macclesfield,permanent,26000
59997,staff nurse nursing home baldock white recru...,baldock,NaN,24500
59998,this is one of the best agency side opportunit...,the city,permanent,65000


In [36]:
vect = TfidfVectorizer(min_df = 5)
X_train_text  = vect.fit_transform(salary_train['FullDescription'])
X_train_text

<60000x22861 sparse matrix of type '<class 'numpy.float64'>'
	with 8365759 stored elements in Compressed Sparse Row format>

In [37]:
salary_train["LocationNormalized"].fillna("nan", inplace=True)
salary_train["ContractTime"].fillna("nan", inplace=True)

In [41]:
enc = DictVectorizer()
X_train_cat = enc.fit_transform(salary_train[["LocationNormalized", "ContractTime"]].to_dict("records"))

In [44]:
X_train = hstack([X_train_text, X_train_cat])
X_train

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [46]:
clf = Ridge(alpha = 1,random_state = 241)
clf.fit(X_train,salary_train['SalaryNormalized'])

Ridge(alpha=1, random_state=241)

In [48]:
test = pd.read_csv('test_salary.csv')
clf.predict()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [49]:
for i in range(len(test.columns)):
    test.iloc[:,i] = test.iloc[:,i].map(lambda x:x if type(x)!=str else x.lower())
    test.iloc[:,i] = test.iloc[:,i].replace('[^a-zA-Z0-9]', ' ', regex = True)
test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,we currently have a vacancy for an hr project ...,milton keynes,contract,NaN
1,a web developer opportunity has arisen with an...,manchester,permanent,NaN


In [51]:
X_test = vect.transform(test['FullDescription'])
X_test_cat = enc.transform(test[["LocationNormalized", "ContractTime"]].to_dict("records"))

In [52]:
X_test = hstack([X_test,X_test_cat])

In [54]:
clf.predict(X_test)

array([56570.91760293, 37139.19471667])